In [1]:
import rasterio as rio
import numpy as np
import numpy.ma as ma
import xarray as xr
import os
from datetime import datetime, timedelta
from pyproj import Proj

#### Point to the directories with ABI data from each satellite and each band we want

In [55]:
filepaths = ['/home/jovyan/goes16/2018/12/12/ABI-L1b-RadC/18/C02/','/home/jovyan/goes17/2018/12/12/ABI-L1b-RadC/18/C02/',
             '/home/jovyan/goes16/2018/12/12/ABI-L1b-RadC/18/C03/','/home/jovyan/goes17/2018/12/12/ABI-L1b-RadC/18/C03/']
file_list = os.listdir(filepaths[2])

# listing the files for GOES-16, band 2 (red)
file_list

['OR_ABI-L1b-RadC-M3C03_G16_s20183461857178_e20183461859551_c20183461859592.nc',
 'OR_ABI-L1b-RadC-M3C03_G16_s20183461842178_e20183461844551_c20183461844594.nc',
 'OR_ABI-L1b-RadC-M3C03_G16_s20183461827178_e20183461829551_c20183461829591.nc',
 'OR_ABI-L1b-RadC-M3C03_G16_s20183461832178_e20183461834551_c20183461834595.nc',
 '.ipynb_checkpoints',
 'OR_ABI-L1b-RadC-M3C03_G16_s20183461852178_e20183461854551_c20183461854593.nc',
 'OR_ABI-L1b-RadC-M3C03_G16_s20183461807178_e20183461809551_c20183461809596.nc',
 'OR_ABI-L1b-RadC-M3C03_G16_s20183461837178_e20183461839551_c20183461839595.nc',
 'OR_ABI-L1b-RadC-M3C03_G16_s20183461822178_e20183461824551_c20183461824593.nc',
 'OR_ABI-L1b-RadC-M3C03_G16_s20183461817178_e20183461819551_c20183461819595.nc',
 'OR_ABI-L1b-RadC-M3C03_G16_s20183461812178_e20183461814551_c20183461814594.nc',
 'OR_ABI-L1b-RadC-M3C03_G16_s20183461847178_e20183461849551_c20183461849594.nc',
 'OR_ABI-L1b-RadC-M3C03_G16_s20183461802178_e20183461804551_c20183461804593.nc']

#### Open NetCDF dataset using xarray

In [56]:
nc_file = filepaths[2] + file_list[2]
conus = xr.open_dataset(nc_file)

#### Look at the GOES Imager Projection information

In [57]:
conus.goes_imager_projection

<xarray.DataArray 'goes_imager_projection' ()>
array(-2147483647, dtype=int32)
Coordinates:
    t        datetime64[ns] ...
    y_image  float32 ...
    x_image  float32 ...
Attributes:
    long_name:                       GOES-R ABI fixed grid projection
    grid_mapping_name:               geostationary
    perspective_point_height:        35786023.0
    semi_major_axis:                 6378137.0
    semi_minor_axis:                 6356752.31414
    inverse_flattening:              298.2572221
    latitude_of_projection_origin:   0.0
    longitude_of_projection_origin:  -75.0
    sweep_angle_axis:                x

***
#### Function for generating a pyproj object (and proj4 string):

In [58]:
def getProj4string(goes_imager_projection):
    '''make a proj4 string  and a pyproj geostationary map object from goes imager projection information'''
    
    h=goes_imager_projection.perspective_point_height #          Satellite height above ellipsoid
    a=goes_imager_projection.semi_major_axis #                   Ellipsoid semi-major axis
    rf=goes_imager_projection.inverse_flattening #               Ellipsoid flattening (describes semi-minor axis)
    lon0=goes_imager_projection.longitude_of_projection_origin # Center longitude (GOES-East ~ -75.5)
    lat0=goes_imager_projection.latitude_of_projection_origin #  Center latitude (0.0)
    sweep=goes_imager_projection.sweep_angle_axis #              "Sweep angle" axis (x)
    
    # Make proj4 string
    proj_string = '"+proj=geos +ellps=GRS80 +h={h} +a={a} +rf={rf} +lon_0={lon0} +lat_0={lat0} +sweep={sweep}"'.format(
        h=h,a=a,rf=rf,lon0=lon0,lat0=lat0,sweep=sweep)
    
    # Use pyproj to define
    p = Proj(proj='geos', h=h, lon_0=lon0, sweep=sweep)
    
    return proj_string, p

In [59]:
# make a proj4 string and a pyproj geostationary map object from this file's projection information
proj_string, p = getProj4string(conus.goes_imager_projection)
print(proj_string)
print(p)

"+proj=geos +ellps=GRS80 +h=35786023.0 +a=6378137.0 +rf=298.2572221 +lon_0=-75.0 +lat_0=0.0 +sweep=x"
pyproj.Proj('+units=m +proj=geos +h=35786023.0 +lon_0=-75.0 +sweep=x ', preserve_units=True)


In [60]:
# Define a window around the Sierras containing Tuolumne Meadows:
# (in degrees latitude and longitude)
xmin = -120
ymax = 38.4
xmax = -118
ymin = 37.4

In [61]:
# use gdal_translate to convert to a GeoTiff including this projection information
tif_file = '{}.tif'.format(nc_file[:-3])
!gdal_translate -of GTiff -a_srs $proj_string NETCDF:$nc_file:Rad  $tif_file

Input file size is 5000, 3000
0...10...20...30...40...50...60...70...80...90...100 - done.


In [62]:
tif_file_cropped = '{}_tuolumne.tif'.format(tif_file[:-4])
!gdalwarp -of GTiff -te $xmin $ymin $xmax $ymax -te_srs EPSG:4326 $tif_file $tif_file_cropped

Creating output file that is 154P x 87L.
Processing /home/jovyan/goes16/2018/12/12/ABI-L1b-RadC/18/C03/OR_ABI-L1b-RadC-M3C03_G16_s20183461827178_e20183461829551_c20183461829591.tif [1/1] : 0Using internal nodata values (e.g. 1023) for image /home/jovyan/goes16/2018/12/12/ABI-L1b-RadC/18/C03/OR_ABI-L1b-RadC-M3C03_G16_s20183461827178_e20183461829551_c20183461829591.tif.
Copying nodata values from source /home/jovyan/goes16/2018/12/12/ABI-L1b-RadC/18/C03/OR_ABI-L1b-RadC-M3C03_G16_s20183461827178_e20183461829551_c20183461829591.tif to destination /home/jovyan/goes16/2018/12/12/ABI-L1b-RadC/18/C03/OR_ABI-L1b-RadC-M3C03_G16_s20183461827178_e20183461829551_c20183461829591_tuolumne.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
